In [1]:
import tensorflow as tf

In [2]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_memory_growth(gpus[0], True)
#   except RuntimeError as e:
#     # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
#     print(e)

### 라이브러리 선언

일반 CPU 활용 시

In [3]:
# # 라이브러리 정의 
# from tensorflow import keras
# from tensorflow.keras import layers
# import numpy as np
# import pandas as pd

cuda nvidia gpu 활용 시

In [4]:
# # How to check if the code is running on GPU or CPU?

# from tensorflow.python.client import device_lib

# print(device_lib.list_local_devices())

# import os
# os.environ["CUDA_VISIBLE_DEVICES"]='1'

intel gpu 활용 시

In [5]:
# import numpy as np
# import os

# # intel gpu 적용
# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
# import keras
# # mnist 데이터셋 
# from keras.datasets import mnist


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
%matplotlib inline

In [7]:
url = '../dataset/pima-indians-diabetes.csv'
data = pd.read_csv(url)
data.head()

,pregnancies,glucose,diastolic,trceps,insulin,bmi,dpf,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# 1. 타입 통합 / 특성 숫자컬럼 추가

### Skip

# 2. 데이터 분리

In [8]:
# 데이터 분리
featuresData = data.drop(columns=['diabetes'])
labelData = data['diabetes']

trainingData_features,\
testData_features,\
trainingData_label,\
testData_label = train_test_split(featuresData,labelData, test_size=0.2, random_state=1)

In [9]:
print(trainingData_features.shape)
print(testData_features.shape)
print(trainingData_label.shape)
print(testData_label.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [10]:
features = list(trainingData_features.select_dtypes(np.number).columns)

In [11]:
features

['pregnancies',
 'glucose',
 'diastolic',
 'trceps',
 'insulin',
 'bmi',
 'dpf',
 'age']

# 3. 모델 생성

In [12]:
modelDim = trainingData_features.reset_index(drop=True).loc[0].shape

In [13]:
model = Sequential()
### 4개의 feature 가 행으로 입력됨
#Hidden Layer-1
unitNo = 8
model.add(Dense(unitNo, activation='relu', input_shape = modelDim))
#Hidden Layer-2
model.add(Dense(unitNo, activation='relu'))
#Output layer
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 모델 시각화
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', 
           show_shapes=True, 
           show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# 4. 모델 컴파일

In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 5. 모델 학습

In [16]:
# 라이브러리 및 로그 디렉토리 정의 join
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from datetime import datetime
import os

logdir = os.path.join("logs_b",datetime.now().strftime("%Y%m%d_%H%M%S"))

In [17]:
# 콜백 옵션 정의
callback_list =[
    EarlyStopping(monitor="val_loss", patience=500),
    TensorBoard(log_dir = logdir),
    ModelCheckpoint(filepath = "./check/model_b_weights.h5", 
                    verbose = 1, 
                    monitor = "val_loss", 
                    save_best_only = True)
]


In [ ]:
# 훈련
history = model.fit( x=trainingData_features, 
            y=trainingData_label,
            batch_size=32,
            epochs=500,
            validation_split=0.2,
            callbacks = callback_list
           )

Epoch 1/500
10/16 [=================>............] - ETA: 0s - loss: 6.2839 - accuracy: 0.6094 
Epoch 1: val_loss improved from inf to 5.17536, saving model to ./check\model_b_weights.h5
16/16 [==============================] - 1s 24ms/step - loss: 6.7333 - accuracy: 0.6049 - val_loss: 5.1754 - val_accuracy: 0.6016
Epoch 2/500
 9/16 [===============>..............] - ETA: 0s - loss: 5.5734 - accuracy: 0.6146
Epoch 2: val_loss improved from 5.17536 to 3.86854, saving model to ./check\model_b_weights.h5
16/16 [==============================] - 0s 12ms/step - loss: 5.2853 - accuracy: 0.6090 - val_loss: 3.8685 - val_accuracy: 0.5854
Epoch 3/500
10/16 [=================>............] - ETA: 0s - loss: 3.8443 - accuracy: 0.6406
Epoch 3: val_loss improved from 3.86854 to 2.75522, saving model to ./check\model_b_weights.h5
16/16 [==============================] - 0s 11ms/step - loss: 3.9937 - accuracy: 0.6110 - val_loss: 2.7552 - val_accuracy: 0.5854
Epoch 4/500
11/16 [===================>....

In [ ]:
# from tensorflow.keras import backend as K

# K.clear_session()
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

In [ ]:
trainingLog  = pd.DataFrame(history.history)

In [ ]:
trainingLog.tail(10)

In [ ]:
model.evaluate(testData_features, testData_label)

In [ ]:
pd.DataFrame(history.history)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(12,8))
# 훈련 데이터의 mse
plt.plot(history.history['accuracy'])
# 검증 데이터의 mse
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy','val_accuracy'])
plt.grid()
plt.show()

In [ ]:
pd.DataFrame(history.history).tail()

# 6. 모델 추론

In [ ]:
import math
# 정확도 추론
model.evaluate(testData_features, testData_label, verbose=1)

# 7. 모델 예측

In [ ]:
# 가상의 환자 데이터 입력
patient_1 = pd.DataFrame([[0,137,90,35,168,43.1,2.288,33]])

# 모델로 예측하기
prediction = model.predict(patient_1)
prediction

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# 예측결과 출력하기
print(prediction*100)

### [실습] 
diabetes 동일 데이터를 활용하여 다음장 학습 종료 후 sigmoid 와 softmax를 활용하여 예측정확도를 비교해보세요.